In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plot
import math

from datetime import timedelta
from datetime import datetime

In [2]:
#id directories
data_dir = os.getcwd()

In [3]:
items_df = pd.read_excel('item_sets.xlsx')
orders_df = pd.read_excel('Critical_Supply_Dashboard_19 items_Aug_19.xlsx', 
                          sheet_name='DATA_Orders')
orders_df.columns = orders_df.iloc[0]
orders_df.columns = orders_df.columns.str.replace(' ', '')
items_df.columns = items_df.columns.str.replace(' ', '')
orders_df.columns = orders_df.columns.str.replace('.', '_')
orders_df = orders_df.iloc[1:]
orders_df = orders_df[orders_df['Category'].notnull()]

In [4]:
#clean orders data frame to get types

orders_df['item_type_list_clean'] = ''
temp = ['PAWS','Disinfectant Wipes', 
        'Hand Sanitizer','Hand Soap',
        'N-95 Filter',
        'Surgical Mask']


for i in range(len(orders_df)):
    
    category_temp = orders_df['Category'].iloc[i]
    
    if category_temp in temp:
        orders_df['item_type_list_clean'].iloc[i] = category_temp
    elif category_temp in ['DS', 'DSS']:
        orders_df['item_type_list_clean'].iloc[i] = 'Disinfecting Solution'
    elif category_temp in ['IA']:
        orders_df['item_type_list_clean'].iloc[i] = 'Isopropyl Alcohol'
    elif ((category_temp.find('Gloves') != -1) and (orders_df['Description'].iloc[i].find('exam') != -1)):
        orders_df['item_type_list_clean'].iloc[i] = 'Nitrile Gloves (Public Safety)'
    elif ((category_temp.find('Gloves') != -1) and (orders_df['Description'].iloc[i].find('exam') == -1)):
        orders_df['item_type_list_clean'].iloc[i] = 'Nitrile Gloves (General Use)'
    elif category_temp in ['CMask']:
        orders_df['item_type_list_clean'].iloc[i] = 'Mask (Cloth)'
    elif category_temp in ['Therm']:
        orders_df['item_type_list_clean'].iloc[i] = 'Thermometer'
    elif (category_temp.find('Tyvek') != -1):
        if (category_temp.find('NH') != -1):
            orders_df['item_type_list_clean'].iloc[i] = 'Tyvek Suits'
        else:
            orders_df['item_type_list_clean'].iloc[i] = 'Tyvek Suits w/ hoods'
    elif category_temp in ['Gowns']:
        orders_df['item_type_list_clean'].iloc[i] = 'Surgical Gowns'
    elif category_temp in ['HS Refill Bags']:
        orders_df['item_type_list_clean'].iloc[i] = 'Purell (1200 ml stand refill)'
    elif (category_temp.find('Toilet') != -1):
        orders_df['item_type_list_clean'].iloc[i] = 'Toilet Paper'
    else:
        orders_df['item_type_list_clean'].iloc[i] = 'NA'
        
orders_df = orders_df[orders_df['item_type_list_clean'] != 'NA']

In [5]:
#delay time df
delay_time_df = orders_df[['item_type_list_clean', 'Date', 'EstimatedDeliveryDate']]

#clean to only include df with estimated/realized delay times
delay_time_df = delay_time_df[delay_time_df['EstimatedDeliveryDate'] != 'TBD']
delay_time_df = delay_time_df[delay_time_df['EstimatedDeliveryDate'].notnull()]
delay_time_df['Date'] = pd.to_datetime(delay_time_df['Date'])
delay_time_df['EstimatedDeliveryDate'] = (delay_time_df['EstimatedDeliveryDate']).astype(str)
delay_time_df['filter'] = delay_time_df['EstimatedDeliveryDate'].str.contains('-') #to filter out other misc errors
delay_time_df = delay_time_df[delay_time_df['filter'] == True]
delay_time_df['EstimatedDeliveryDate'] = pd.to_datetime(delay_time_df['EstimatedDeliveryDate'])
delay_time_df['delay_time'] = delay_time_df['EstimatedDeliveryDate'] - delay_time_df['Date']